<a href="https://colab.research.google.com/github/hwon-n/cp2/blob/hyewon/fine_tuning_bert_with_mxnet_gluon2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT fine funing

[참고](https://www.kaggle.com/code/jt0321/fine-tuning-bert-with-mxnet-gluon/notebook)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


`!nvcc --version` 으로 cuda 버전 확인 후 [mxnet](https://pypi.org/project/mxnet/) 들어가서 맞는 mxnet-cu1xx 설치하기

In [4]:
!pip install mxnet-cu110

     |████████████████████████████████| 325.4 MB 29 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [24]:
pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-br65gna3
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-br65gna3
     |████████████████████████████████| 132 kB 8.7 MB/s 
     |████████████████████████████████| 47.3 MB 1.8 MB/s 
     |████████████████████████████████| 4.5 MB 36.5 MB/s 
     |████████████████████████████████| 3.8 MB 57.5 MB/s 
     |████████████████████████████████| 6.5 MB 50.3 MB/s 
     |████████████████████████████████| 67 kB 6.1 MB/s 
     |████████████████████████████████| 895 kB 61.2 MB/s 
     |████████████████████████████████| 596 kB 63.9 MB/s 
     |████████████████████████████████| 8.6 MB 36.4 MB/s 
     |████████████████████████████████| 79 kB 9.4 MB/s 
     |████████████████████████████████| 127 kB 76.2 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=9c87c0168600343ce1ca77183708a62e59819732e98c56102006cba8c0125973
  Stor

/usr/local/lib/python3.7/dist-packages/google/colab/_pip.py:87: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.7/dist-packages/boto3-1.21.27.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):
/usr/local/lib/python3.7/dist-packages/google/colab/_pip.py:87: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.7/dist-packages/botocore-1.24.27.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):
/usr/local/lib/python3.7/dist-packages/google/colab/_pip.py:87: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.7/dist-packages/huggingface_hub-0.4.0.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):
/usr/local/lib/python3.7/dist-packages/google/colab/_pip.py:87: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.7/dist-packages/jmespath-1.0.0.dist-info/top_level.txt' mode='r' encoding='UTF-8'

In [15]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 7.0 MB/s 


/usr/local/lib/python3.7/dist-packages/google/colab/_pip.py:87: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.7/dist-packages/sentencepiece-0.1.96.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):


In [5]:
!pip install gluonnlp

     |████████████████████████████████| 344 kB 8.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595733 sha256=9d113bfef5abfc46227494fe835819229ff65acb6d42801d8b8e04fa202c6575
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp


In [6]:
import warnings
warnings.filterwarnings(action='once')

import os
from pathlib import Path
import random
import multiprocessing

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook, tqdm

import mxnet as mx
from mxnet import gluon
from mxnet.gluon import Block
from mxnet.gluon import nn
import gluonnlp as nlp
from gluonnlp.data import BERTTokenizer, BERTSentenceTransform

/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/lib/python3.7/dist-packages/mxnet/numpy/utils.py:37: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence th

In [7]:
train_tsv = '/content/drive/MyDrive/news_class9x13000/train.tsv'
test_tsv = '/content/drive/MyDrive/news_class9x13000/test.tsv'

In [8]:
train_df = pd.read_csv(train_tsv, delimiter = '\t', header = None)
train_df.columns = ['target', 'prompt', 'next']
train_df.head()

,target,prompt,next
0,1,프로크루스테스의 침대라는 그리스신화에 나오는 이야기가 있다.,이 이야기는 프로크루스테스라는 사람이 지나가는 나그네를 자신의 집으로 잡아와서 침대...
1,0,이 이야기는 프로크루스테스라는 사람이 지나가는 나그네를 자신의 집으로 잡아와서 침대...,"또, 안전행정부와 기획재정부는 전 부처와 공공기관을 대상으로 여름휴가 하루 더 가기..."
2,0,결국 침대에 사람을 맞춘다는 얘기다.,올해부터 실시하고 있는 활동보조서비스 개악 지침도 이것과 다르지 않게 잔인하다.
3,0,올해부터 실시하고 있는 활동보조서비스 개악 지침도 이것과 다르지 않게 잔인하다.,흥선대원군은 이홍장의 심문을 받은 후 바오딩(保定)으로 옮겨져 4년간 중국에서 불우...
4,1,장애인활동보조서비스는 장애로 인해 일상생활의 어려움을 겪는 사람들의 원활한 일상생활...,2007년부터 시작한 이 서비스 제도는 전국의 중증장애인들이 국가를 상대로 치열한 ...


In [9]:
test_df = pd.read_csv(test_tsv, delimiter = '\t', header = None)
test_df.columns = ['target', 'prompt', 'next']
test_df.head()

,target,prompt,next
0,1,보름달 뜨면 구름 자주 끼고,꽃이 활짝 피면 바람이 불어대지.
1,0,꽃이 활짝 피면 바람이 불어대지.,◈ 안쓰고 쌓아두면 세금...기업소득환류세제
2,1,세상일이란 모두 이런 거야.,나 홀로 웃는 까닭 아는 이 없을걸.
3,0,나 홀로 웃는 까닭 아는 이 없을걸.,권력기관에서 경력을 쌓은 인물들도 적지 않다.
4,1,정약용이 지은 독소(獨笑·홀로 웃다)라는 시이다.,이렇게 인간사 누구나 한 가지씩 걱정거리를 안고 사는 게 우리네 인생이다.


### classes

In [10]:
class BERTClassifier(Block):
    """Model for sentence (pair) classification task with BERT.

    The model feeds token ids and token type ids into BERT to get the
    pooled BERT sequence representation, then apply a Dense layer for
    classification.

    Parameters
    ----------
    bert: BERTModel
        Bidirectional encoder with transformer.
    num_classes : int, default is 2
        The number of target classes.
    dropout : float or None, default 0.0.
        Dropout probability for the bert output.
    prefix : str or None
        See document of `mx.gluon.Block`.
    params : ParameterDict or None
        See document of `mx.gluon.Block`.
    """

    def __init__(self,
                 bert,
                 num_classes=2,
                 dropout=0.0,
                 prefix=None,
                 params=None):
        super(BERTClassifier, self).__init__(prefix=prefix, params=params)
        self.bert = bert
        with self.name_scope():
            self.classifier = nn.HybridSequential(prefix=prefix)
            if dropout:
                self.classifier.add(nn.Dropout(rate=dropout))
            self.classifier.add(nn.Dense(units=num_classes))

    def forward(self, inputs, token_types, valid_length=None):  # pylint: disable=arguments-differ
        """Generate the unnormalized score for the given the input sequences.

        Parameters
        ----------
        inputs : NDArray, shape (batch_size, seq_length)
            Input words for the sequences.
        token_types : NDArray, shape (batch_size, seq_length)
            Token types for the sequences, used to indicate whether the word belongs to the
            first sentence or the second one.
        valid_length : NDArray or None, shape (batch_size)
            Valid length of the sequence. This is used to mask the padded tokens.

        Returns
        -------
        outputs : NDArray
            Shape (batch_size, num_classes)
        """
        _, pooler_out = self.bert(inputs, token_types, valid_length)
        return self.classifier(pooler_out)

In [11]:
class BERTDatasetTransform(object):
    """Dataset Transformation for BERT-style Sentence Classification or Regression.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    labels : list of int , float or None. defaults None
        List of all label ids for the classification task and regressing task.
        If labels is None, the default task is regression
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    label_dtype: int32 or float32, default float32
        label_dtype = int32 for classification task
        label_dtype = float32 for regression task
    """

    def __init__(self,
                 tokenizer,
                 max_seq_length,
                 class_labels=None,
                 pad=True,
                 pair=True,
                 has_label=True):
        self.class_labels = class_labels
        self.has_label = has_label
        self._label_dtype = 'int32' if class_labels else 'float32'
        if has_label and class_labels:
            self._label_map = {}
            for (i, label) in enumerate(class_labels):
                self._label_map[label] = i
        self._bert_xform = BERTSentenceTransform(
            tokenizer, max_seq_length, pad=pad, pair=pair)

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
          sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 3 strings:
        text_a, text_b and label.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
            label: '0'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens:  '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14
            label: 0

        For single sequences, the input is a tuple of 2 strings: text_a and label.
        Inputs:
            text_a: 'the dog is hairy .'
            label: '1'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a:  '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7
            label: 1

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 3 strings:
            (text_a, text_b, label). For single sequences, the input is a tuple
            of 2 strings: (text_a, label).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)
        np.array: classification task: label id in 'int32', shape (batch_size, 1),
            regression task: label in 'float32', shape (batch_size, 1)
        """
        if self.has_label:
            input_ids, valid_length, segment_ids = self._bert_xform(line[:-1])
            label = line[-1]
            # map to int if class labels are available
            if self.class_labels:
                label = self._label_map[label]
            label = np.array([label], dtype=self._label_dtype)
            return input_ids, valid_length, segment_ids, label
        else:
            return self._bert_xform(line)


### preprocessing

In [12]:
ctx = mx.gpu()

In [13]:
max_len = 128
pad = True
pair = True

epochs = 3
batch_size = 32
#optimizer='bertadam'
class_labels=[0, 1]
lr = 1e-6
epsilon = 1e-06
warmup_ratio = 0.1
log_interval = 1000
accumulate = None

In [30]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12', 
                                             dataset_name='kobert_news_wiki_ko_cased',
                                             pretrained=True, ctx=ctx, use_pooler=True,
                                             use_decoder=False, use_classifier=False,
                                             root='/content/drive/MyDrive/bert-mx')
print(bert_base)

BERTModel(
  (encoder): BERTEncoder(
    (dropout_layer): Dropout(p = 0.1, axes=())
    (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
    (transformer_cells): HybridSequential(
      (0): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (attention_cell): DotProductSelfAttentionCell(
          (dropout_layer): Dropout(p = 0.1, axes=())
        )
        (proj): Dense(768 -> 768, linear)
        (ffn): PositionwiseFFN(
          (ffn_1): Dense(768 -> 3072, linear)
          (activation): GELU()
          (ffn_2): Dense(3072 -> 768, linear)
          (dropout_layer): Dropout(p = 0.1, axes=())
          (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
        )
        (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
      )
      (1): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (attention_cell): DotProductSelfAttention

In [32]:
bert_classifier = nlp.model.BERTClassifier(bert_base, num_classes=2, dropout=0.1)
# only need to initialize the classifier layer.
bert_classifier.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
bert_classifier.hybridize(static_alloc=True)

# softmax cross entropy loss for classification
loss_function = gluon.loss.SoftmaxCELoss()
loss_function.hybridize(static_alloc=True)

metric = mx.metric.Accuracy()

In [33]:
tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=False)

In [34]:
train_df['prompt'] = train_df['prompt'].astype(str)
train_df['next'] = train_df['next'].astype(str)
train_df['target']=train_df['target'].astype(int)

In [35]:
train_data_raw = train_df[['prompt', 'next', 'target']].values

In [36]:
pool = multiprocessing.Pool()

# transformation for data train and dev
#label_dtype = 'float32' if not task.class_labels else 'int32'
label_dtype='int32'
trans = BERTDatasetTransform(tokenizer, max_len,
                             class_labels=class_labels,
                             pad=pad, pair=pair,
                             has_label=True)

# data train
# task.dataset_train returns (segment_name, dataset)
#train_tsv = task.dataset_train()[1]
#data_train = mx.gluon.data.SimpleDataset(pool.map(trans, train_data_raw))
data_train = mx.gluon.data.SimpleDataset(train_data_raw)
data_train = data_train.transform(trans)
data_train_len = data_train.transform(
    lambda input_id, length, segment_id, label_id: length, lazy=False)
# bucket sampler for training
batchify_fn = nlp.data.batchify.Tuple(
    nlp.data.batchify.Pad(axis=0), nlp.data.batchify.Stack(),
    nlp.data.batchify.Pad(axis=0), nlp.data.batchify.Stack(label_dtype))
batch_sampler = nlp.data.sampler.FixedBucketSampler(
    data_train_len,
    batch_size=batch_size,
    #num_buckets=10,
    ratio=0,
    shuffle=True)
# data loader for training
train_data = gluon.data.DataLoader(
    dataset=data_train,
    num_workers=4,
    batch_sampler=batch_sampler,
    batchify_fn=batchify_fn)
num_train_examples = len(data_train)

/usr/local/lib/python3.7/dist-packages/gluonnlp/data/batchify/batchify.py:235: UserWarning: Padding value is not given and will be set automatically to 0 in data.batchify.Pad(). Please check whether this is intended (e.g. value of padding index in the vocabulary).
  'Padding value is not given and will be set automatically to 0 '


In [37]:
print('vocabulary used for tokenization = \n%s'%vocabulary)
print('[PAD] token id = %s'%(vocabulary['[PAD]']))
print('[CLS] token id = %s'%(vocabulary['[CLS]']))
print('[SEP] token id = %s'%(vocabulary['[SEP]']))
print('token ids = \n%s'%data_train[4][0])
print('valid length = \n%s'%data_train[4][1])
print('segment ids = \n%s'%data_train[4][2])
print('label = \n%s'%data_train[4][3])

vocabulary used for tokenization = 
Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
[PAD] token id = 1
[CLS] token id = 2
[SEP] token id = 3
token ids = 
[ 2  0  0  0  0  0  0  0  0  0  0  0  0  0 54  3  0  0  0  0  0  0  0  0
  0  0  0  0  0  0 54  3  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1]
valid length = 
32
segment ids = 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
label = 
[1]


### training

In [38]:
optimizer = mx.optimizer.create('adam', learning_rate=lr, epsilon=epsilon)

In [39]:
all_model_params = bert_classifier.collect_params()
optimizer_params = {'learning_rate': lr, 'epsilon': epsilon, 'wd': 0.01}

try:
    trainer = gluon.Trainer(all_model_params, optimizer,
                            update_on_kvstore=False)
except ValueError as e:
    print(e)
    warnings.warn(
        'AdamW optimizer is not found. Please consider upgrading to '
        'mxnet>=1.5.0. Now the original Adam optimizer is used instead.')
    trainer = gluon.Trainer(all_model_params, 'adam',
                            optimizer_params, update_on_kvstore=False)

In [40]:
# bert_classifier.load_parameters('/content/drive/MyDrive/bert-mx/j-20190616k', ctx=ctx)

In [41]:
step_size = batch_size * accumulate if accumulate else batch_size
num_train_steps = int(num_train_examples / step_size * epochs)
num_warmup_steps = int(num_train_steps * warmup_ratio)
step_num = 0

# Do not apply weight decay on LayerNorm and bias terms
for _, v in bert_classifier.collect_params('.*beta|.*gamma|.*bias').items():
    v.wd_mult = 0.0
# Collect differentiable parameters
params = [p for p in all_model_params.values() if p.grad_req != 'null']

# Set grad_req if gradient accumulation is required
if accumulate:
    for p in params:
        p.grad_req = 'add'

#tic = time.time()
for epoch_id in range(epochs):
    metric.reset()
    step_loss = 0
    #tic = time.time()
    all_model_params.zero_grad()

    for batch_id, seqs in enumerate(train_data):
        # learning rate schedule
        if step_num < num_warmup_steps:
            new_lr = lr * step_num / num_warmup_steps
        else:
            non_warmup_steps = step_num - num_warmup_steps
            offset = non_warmup_steps / (num_train_steps - num_warmup_steps)
            new_lr = lr - offset * lr
        optimizer.set_learning_rate(new_lr)

        # forward and backward
        with mx.autograd.record():
            input_ids, valid_length, type_ids, label = seqs
            out = bert_classifier(
                input_ids.as_in_context(ctx), type_ids.as_in_context(ctx),
                valid_length.astype('float32').as_in_context(ctx))
            ls = loss_function(out, label.as_in_context(ctx)).mean()
        ls.backward()

        # update
        if not accumulate or (batch_id + 1) % accumulate == 0:
            trainer.allreduce_grads()
            nlp.utils.clip_grad_global_norm(params, 1)
            trainer.update(accumulate if accumulate else 1)
            # set grad to zero for gradient accumulation
            all_model_params.zero_grad()
            step_num += 1

        step_loss += ls.asscalar()
        metric.update([label], [out])
        if (batch_id + 1) % (100) == 0:
            #log_train(batch_id, len(train_data), metric, step_loss, log_interval,
                      #epoch_id, trainer.learning_rate)
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f}'
                  .format(epoch_id, batch_id + 1, len(train_data),
                 step_loss / log_interval,
                 optimizer.learning_rate, metric.get()[1]))
            step_loss = 0
    mx.nd.waitall()

[Epoch 0 Batch 100/474] loss=0.0697, lr=0.0000007, acc=0.498
[Epoch 0 Batch 200/474] loss=0.0690, lr=0.0000010, acc=0.521
[Epoch 0 Batch 300/474] loss=0.0686, lr=0.0000009, acc=0.533
[Epoch 0 Batch 400/474] loss=0.0689, lr=0.0000008, acc=0.538
[Epoch 1 Batch 100/474] loss=0.0683, lr=0.0000007, acc=0.569
[Epoch 1 Batch 200/474] loss=0.0686, lr=0.0000006, acc=0.559
[Epoch 1 Batch 300/474] loss=0.0683, lr=0.0000005, acc=0.561
[Epoch 1 Batch 400/474] loss=0.0683, lr=0.0000004, acc=0.560
[Epoch 2 Batch 100/474] loss=0.0680, lr=0.0000003, acc=0.565
[Epoch 2 Batch 200/474] loss=0.0680, lr=0.0000002, acc=0.569
[Epoch 2 Batch 300/474] loss=0.0679, lr=0.0000001, acc=0.569
[Epoch 2 Batch 400/474] loss=0.0680, lr=0.0000000, acc=0.567


In [24]:
bert_classifier.save_parameters('/content/drive/MyDrive/bert-mx/j-20220326k')

In [51]:
# The hyperparameters
batch_size = 32
lr = 5e-6

# The FixedBucketSampler and the DataLoader for making the mini-batches
train_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[1]) for item in data_train],
                                            batch_size=batch_size,
                                            shuffle=True)
bert_dataloader = mx.gluon.data.DataLoader(data_train, batch_sampler=train_sampler)

trainer = mx.gluon.Trainer(bert_classifier.collect_params(), 'adam',
                           {'learning_rate': lr, 'epsilon': 1e-9})

# Collect all differentiable parameters
# `grad_req == 'null'` indicates no gradients are calculated (e.g. constant parameters)
# The gradients for these params are clipped later
params = [p for p in bert_classifier.collect_params().values() if p.grad_req != 'null']
grad_clip = 1

# Training the model with only three epochs
log_interval = 4
num_epochs = 3
for epoch_id in range(num_epochs):
    metric.reset()
    step_loss = 0
    for batch_id, (token_ids, segment_ids, valid_length, label) in enumerate(bert_dataloader):
        with mx.autograd.record():

            # Load the data to the GPU
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # Forward computation
            out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
            ls = loss_function(out, label).mean()

        # And backwards computation
        ls.backward()

        # Gradient clipping
        trainer.allreduce_grads()
        nlp.utils.clip_grad_global_norm(params, 1)
        trainer.update(1)

        step_loss += ls.asscalar()
        metric.update([label], [out])

        # Printing vital information
        if (batch_id + 1) % (log_interval) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f}'
                         .format(epoch_id, batch_id + 1, len(bert_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
            step_loss = 0

MXNetError: ignored

### prediction

In [42]:
test_df['prompt'] = test_df['prompt'].astype(str)
test_df['next'] = test_df['next'].astype(str)
test_data_raw = test_df[['prompt', 'next']].values

In [43]:
# batchify for data test
test_batchify_fn = nlp.data.batchify.Tuple(
    nlp.data.batchify.Pad(axis=0), nlp.data.batchify.Stack(),
    nlp.data.batchify.Pad(axis=0))

# transform for data test
test_trans = BERTDatasetTransform(tokenizer, max_len,
                                  class_labels=None,
                                  pad=pad, pair=pair,
                                  has_label=False)

data_test = mx.gluon.data.SimpleDataset(pool.map(test_trans, test_data_raw))
loader_test = mx.gluon.data.DataLoader(
    data_test,
    batch_size=batch_size,
    num_workers=4,
    shuffle=False,
    batchify_fn=test_batchify_fn)

/usr/local/lib/python3.7/dist-packages/gluonnlp/data/batchify/batchify.py:235: UserWarning: Padding value is not given and will be set automatically to 0 in data.batchify.Pad(). Please check whether this is intended (e.g. value of padding index in the vocabulary).
  'Padding value is not given and will be set automatically to 0 '


In [44]:
data_test[0]

(array([ 2,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0, 54,  3,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1], dtype=int32),
 array(14, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 

In [45]:
#value_list = []
#index_list = []
results = []
for _, seqs in enumerate(loader_test):
    input_ids, valid_length, type_ids = seqs
    out = bert_classifier(input_ids.as_in_context(ctx),
                type_ids.as_in_context(ctx),
                valid_length.astype('float32').as_in_context(ctx))
    results.extend([o for o in out.asnumpy()])
    #values, indices = mx.nd.topk(out, k=1, ret_typ='both')
    #value_list.extend(values.asnumpy().reshape(-1).tolist())
    #index_list.extend(indices.asnumpy().reshape(-1).tolist())

mx.nd.waitall()



In [46]:
results[24]

array([ 0.02147342, -0.07293352], dtype=float32)

In [47]:
predictions = [mx.nd.array(result).sigmoid().asnumpy()[1] for result in results]

In [48]:
submission = pd.DataFrame.from_dict({
    'prompt': test_df['prompt'],
    'next': test_df['next'],
    'prediction': predictions
})

In [49]:
submission.to_csv('/content/drive/MyDrive/data/submission.csv', index=False)